In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import scipy.interpolate as spi
import scipy.optimize as spo
import scipy.stats as sps
import git
import altair as alt
import hysttools as ht
from pathlib import Path
from datetime import date

In [ ]:
from altair_data_server import data_server
alt.data_transformers.enable('data_server')

In [ ]:
repo=git.Repo('.', search_parent_directories=True)
path_root = Path(repo.working_tree_dir)
data_folder = path_root / 'data'
processed_folder = data_folder / 'processed'
vsm_folder = data_folder / 'raw'/'vsm'
image_folder = data_folder / 'images'
search_pattern = re.compile('.*hysu$') # This is the suffix for a hysteresis file

file_list = list(filter(search_pattern.match,os.listdir(vsm_folder)))
specimens=pd.DataFrame(data={'name':file_list,'file':file_list})
specimens['name']=specimens['name'].str.replace('\.hysu','').str.lower()

location_names={'.*eg.*':'glacier','.*cb.*':'bay','.*combay.*':'bay','.*suspended.*':'suspended','.*bed.*':'bed'}
specimens['location']=specimens['name'].replace(location_names.keys(),location_names.values(),regex=True)

print(specimens)
print("\n{} HYSU files to process.".format(np.shape(specimens)[0]))

In [ ]:
Hgrid=ht.make_hgrid(200,0.8,0.5)
Hgrid2=ht.make_hgrid2(200,0.8)
hyst_rh=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
hyst_ih=pd.DataFrame(columns=['id']+["{}".format(x) for x in Hgrid])
all_irm=pd.DataFrame(columns=['id','field','remanence','gradient'])

In [ ]:
file_skip=[]
hyst_stats=pd.DataFrame(columns=['id','Mrs','Ms','Hc','Xhf','Q','Qeff'])

for index,row in specimens.iterrows():
    name,file_name,location = row
    file_path = vsm_folder / file_name
    print('Working on {}\n{}\n'.format(file_name,file_path))
    if (file_name in file_skip):
        print('Skipping... {}\n'.format(file_name))
    if (file_name not in file_skip):
        print("Processing: {}\n".format(name))
        s,d=ht.read_hyst(file_path)
        print("Hysteresis file read with {} data points.".format(d['moment'].shape[0]))
        result=ht.process_hyst(s,d,Hgrid=Hgrid)
        print("Hysteresis file processed.")
        fig=plt.figure(1)
        ax_tuple=ht.plot_hyst_report(result,fig,name)
        plt.savefig(image_folder/'{}_hystreport.svg'.format(name),dpi=300)
        plt.savefig(image_folder/'{}_hystreport.png'.format(name),dpi=300)
        to_add=pd.Series({'id':name,
                                        'Mrs':float(result['Mrs']),
                                        'Ms':float(result['Ms']),
                                        'Hc':float(result['Hc']),
                                        'Xhf':float(result['Xhf']),
                                        'Q':float(result['Q']),
                                        'Qeff':float(result['Qeff'])})
        hyst_stats=hyst_stats.append(to_add,ignore_index=True)
        to_add=pd.Series(dict(zip(["{}".format(x) for x in result['reversible']['field']],result['reversible']['moment'])))
        to_add['id']=name
        hyst_rh=hyst_rh.append(to_add,ignore_index=True)
        to_add=pd.Series(dict(zip(["{}".format(x) for x in result['irreversible']['field']],result['irreversible']['moment'])))
        to_add['id']=name
        hyst_ih=hyst_ih.append(to_add,ignore_index=True)
        

In [ ]:
hyst_stats['MrsMs']=hyst_stats['Mrs']/hyst_stats['Ms']
hyst_stats

In [ ]:
hyst_stats.to_csv(processed_folder/'hyst_stats.csv')

In [ ]:
hyst_chart=alt.Chart(hyst_stats).mark_point().encode(
    x='Hc:Q',
    y='MrsMs:Q',
    size='Q:Q',
    color='Xhf:Q'
)
display(hyst_chart)